In [18]:
!pip install tiktoken -q

In [6]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OPENAI_API_KEY")

OPENAI_API_KEY ········


In [7]:
from langchain.llms import OpenAI

model = OpenAI(temperature=0.6)

In [8]:
from langchain.document_loaders import TextLoader
loader = TextLoader('transcript.txt')
documents = loader.load()


In [9]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(model, chain_type="map_reduce", verbose=True)
caption = chain.run(documents)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


" What's up everyone? In this video I show you how to do file handling and file organization in Python. So I show you how to navigate to different directories how to rename files, move them, copy them or remove files and directories. So these are all the functions we need in order to automate our file organization. And then I also show you one real life example. So I show you one automation script that I use in order to keep my desktop organized. So let's get started. First of all, let's learn how we can navigate to a different directory and for this we use the OS module and first of all we can print the current working directory by calling os.getcvd and this will print this path so this is where the main.py file is located and now let's switch to a different directory. So here on the desktop I have one folder that I called video file

In [10]:
caption

'\nThis video tutorial teaches the viewer how to do file handling and organization in Python, including how to navigate, rename, move, copy and remove files and directories. An example of a real-life automation script is then demonstrated, and the viewer is shown how to set up a cron job to automate the process.'

In [21]:
prefix = """You are an AI assistant which generates a summary from a transcript text of a video. \
The following is the transcript text generated from a video. Creating meaningful summary of the \
the transcript is your job. Based on the summary, anyone can learn the topics covered in the video. 
The summary should not be just a caption which only describes about the content of the video. You can create \
seperate points to demonstrate topics coverd in the video and which all things are discussed in that topic; write that inside \
that particular points. In short, the summary should be precised and descriptive unlike a shorter caption.
"""

suffix = """
Transcript: {text}
Summary: """



In [22]:
from src.prompts.video_summarizer_prompt import VideoSummarizerPromptTemplate

prompt = VideoSummarizerPromptTemplate(
    prefix=prefix,
    suffix = suffix,
    input_variables = ["text"]
)

In [23]:
prompt_template = prompt.format(
    text = documents[0].page_content
)

In [24]:
print(prompt_template)

You are an AI assistant which generates a summary from a transcript text of a video. The following is the transcript text generated from a video. Creating meaningful summary of the the transcript is your job. Based on the summary, anyone can learn the topics covered in the video. 
The summary should not be just a caption which only describes about the content of the video. You can create seperate points to demonstrate topics coverd in the video and which all things are discussed in that topic; write that inside that particular points. In short, the summary should be precised and descriptive unlike a shorter caption.


Transcript:  What's up everyone? In this video I show you how to do file handling and file organization in Python. So I show you how to navigate to different directories how to rename files, move them, copy them or remove files and directories. So these are all the functions we need in order to automate our file organization. And then I also show you one real life example

In [31]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm = model,
    prompt = prompt
)


In [32]:
print(chain.run(documents[0].page_content))


This video covers how to do file handling and file organization in Python. It explains how to navigate to different directories, rename files, move them, copy them, and remove files and directories. It also provides a real-life example of an automation script used to keep the desktop organized. This includes how to split the base name from the extension, modify the name, and create a new name. Moreover, it teaches how to move and copy files with the modern Python way using the pathlib module. Lastly, it provides a backup script to keep the desktop clean and how to run it as a cron job.
